In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [ ]:
import os
print(os.getenv("PINECONE_API_KEY"))  # Check if the output is None or your actual API key


23b9a516-ad59-4686-9e1f-2897d20fc448


In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name = "rag",
    dimension = 1536,
    metric = "cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [ ]:
import json
data = json.load(open("reviews.json")) # open the json file
data['reviews'] # get the reviews object from within the json file

[{'professor': 'Dr. Emily Robertson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Very detailed and clear in explanations, though the exams are tough.'},
 {'professor': 'Dr. Mark Johnson',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Inspiring lectures and genuinely cares about student success.'},
 {'professor': 'Dr. Anna Kowalski',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Good course material, but lectures can be a bit monotonous.'},
 {'professor': 'Dr. John Lee',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Passionate about the subject, makes learning enjoyable.'},
 {'professor': 'Dr. Clara Zheng',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Challenging projects but very rewarding, great at explaining difficult concepts.'},
 {'professor': 'Dr. David Smith',
  'subject': 'History',
  'stars': 2,
  'review': 'Needs to update his materials, but fairly knowledgeable.'},
 {'professor': 'Dr. Linda Bates',
  'subject': 'English Literature',

In [ ]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [ ]:
processed_data[0]

{'values': [-0.04996286,
  -0.009815108,
  0.030469153,
  -0.00087622704,
  -0.0007819495,
  -0.013705659,
  -0.030250737,
  0.026373837,
  0.008142854,
  0.042290967,
  0.013378033,
  -0.00157755,
  -0.03380001,
  -0.038277555,
  -0.00814968,
  0.0056959023,
  0.0021773432,
  0.00048290606,
  0.018934013,
  0.015766967,
  0.067873046,
  -0.026878927,
  0.025691284,
  0.020258164,
  -0.023466162,
  -0.031506635,
  -0.002865015,
  0.021445807,
  0.020026097,
  0.015712364,
  0.072077565,
  -0.025309054,
  0.031042498,
  -0.039888382,
  -0.024530945,
  0.039178528,
  0.029841205,
  0.045621824,
  0.013808042,
  0.015234577,
  0.007815229,
  0.041116975,
  -0.03207998,
  -0.031861562,
  0.035301626,
  0.017951136,
  -0.04428402,
  -0.020121654,
  0.023698231,
  0.012661353,
  -0.02823038,
  0.07153153,
  0.08075964,
  0.019084174,
  -0.038523275,
  -0.020490233,
  0.039069317,
  0.02096802,
  0.013186919,
  -0.009712725,
  0.070166424,
  -0.045266896,
  0.026346534,
  -0.04376528,
  -0.01

In [ ]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}